# Agents Azure AI avec prise en charge du protocole de contexte de modèle (MCP)

Ce notebook montre comment utiliser les agents Azure AI avec les outils du protocole de contexte de modèle (MCP) pour créer un agent intelligent capable de tirer parti de serveurs MCP externes pour des capacités améliorées.


## Installer les packages NuGet requis

Tout d'abord, nous devons installer le package Azure AI Agents Persistent, qui fournit les fonctionnalités principales pour travailler avec les agents Azure AI.


## Avantages de l'authentification sans clé

Ce notebook illustre les avantages de l'**authentification sans clé** :
- ✅ **Pas de clés API à gérer** - Utilise l'authentification basée sur l'identité Azure
- ✅ **Sécurité renforcée** - Aucun secret stocké dans le code ou la configuration
- ✅ **Rotation automatique des identifiants** - Azure gère le cycle de vie des identifiants
- ✅ **Accès basé sur les rôles** - Utilise Azure RBAC pour des permissions granulaires

Le `DefaultAzureCredential` utilisera automatiquement la meilleure source d'identifiants disponible :
1. Identité managée (lors de l'exécution dans Azure)
2. Identifiants Azure CLI (pendant le développement)
3. Identifiants Visual Studio
4. Variables d'environnement (si configurées)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installez le package Azure Identity pour l'authentification avec les services Azure en utilisant DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importer les espaces de noms requis

Importez les espaces de noms nécessaires pour Azure AI Agents et Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configurer le client Azure AI Agent (Authentification sans clé)

Configurez les variables de configuration et créez le PersistentAgentsClient en utilisant **l'authentification sans clé** :
- **projectEndpoint** : Le point de terminaison du projet Azure AI Foundry
- **modelDeploymentName** : Le nom du modèle d'IA déployé (GPT-4.1 nano)
- **mcpServerUrl** : L'URL du serveur MCP (API Microsoft Learn)
- **mcpServerLabel** : Un label pour identifier le serveur MCP
- **DefaultAzureCredential** : Utilise l'identité managée, Azure CLI ou d'autres sources d'identification (aucune clé API requise)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Créer une définition d'outil MCP

Créez une définition d'outil MCP qui se connecte au serveur MCP de Microsoft Learn. Cela permettra à l'agent d'accéder au contenu et à la documentation de Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Créer l'agent IA

Créez un agent IA persistant avec le modèle spécifié et les outils MCP. L'agent est configuré avec :
- Le modèle GPT-4.1 nano  
- Des instructions pour utiliser les outils MCP comme assistance  
- Un accès au serveur MCP de Microsoft Learn  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Créer un fil de discussion et envoyer un message

Créez un fil de conversation et envoyez un message utilisateur demandant la différence entre Azure OpenAI et OpenAI. Cela permettra de tester la capacité de l'agent à utiliser les outils MCP pour fournir des informations précises.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configurer les ressources de l'outil MCP (sans clé)

Configurez les ressources de l'outil MCP. Pour une approche véritablement sans clé, vous pouvez supprimer les en-têtes personnalisés si le serveur MCP prend en charge l'authentification basée sur l'identité Azure. L'exemple ci-dessous montre comment ajouter des en-têtes si nécessaire, mais pour les scénarios sans clé, ceux-ci peuvent ne pas être requis.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Démarrer l'exécution de l'agent

Créez et démarrez une exécution pour traiter le message de l'utilisateur. L'agent utilisera les outils et ressources MCP configurés pour générer une réponse.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Surveiller l'exécution et gérer les approbations d'outils (sans clé)

Surveillez l'état d'exécution de l'agent et gérez les approbations nécessaires des outils. Cette boucle :
1. Attend que l'exécution soit terminée ou nécessite une action
2. Approuve automatiquement les appels d'outils MCP lorsque cela est requis
3. Pour l'authentification sans clé, les en-têtes peuvent ne pas être nécessaires si le serveur MCP prend en charge l'identité Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Afficher les Résultats de la Conversation

Récupérez et affichez tous les messages du fil de discussion, en montrant à la fois la question de l'utilisateur et la réponse de l'agent. Les messages sont affichés dans l'ordre chronologique avec des horodatages et des indicateurs de rôle.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
